## Functionality to combine visualizations

In [ ]:
import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline  
from models.binary_model.binary_model import BinaryModel
from models.ind_model.ind_model import OvAModel
from models.multi_model.multi_model import MultiModel

from matplotlib import rc
import matplotlib as mpl

from mainmodel.helper_plotting import *

mags = ["g_mag",  "r_mag", "i_mag", "z_mag", "y_mag",
        "W1_mag", "W2_mag",
        "J_mag", "K_mag", "H_mag"]


exp_dir = "../../../experiments/paper_set/v3/"

In [ ]:
# Multi 
multi_model = MultiModel(folds = 10, min_class_size = 50,  
                   transform_features = True, cols = mags) 
multi_model=load_prev_exp(exp_dir, "multi/Multiclass_Classifier1/", multi_model)

In [ ]:

  

# Binary
model2 = BinaryModel(folds = 10, min_class_size = 50,  
                     transform_features = True, cols = mags) 
binary_model = load_prev_exp(exp_dir, "binary/Binary_Classifiers1/", model2)


# OvA 
model = OvAModel(folds = 10, 
                  init_from_binary = binary_model,
                  min_class_size = 50,  
                   transform_features = True, 
                  cols = mags)  

ova_model = load_prev_exp(exp_dir, "ova/OVA_Classifier1/", model) 

In [ ]:
model = binary_model
from mainmodel.helper_compute import *

N = model.num_runs if model.num_runs is not None else model.num_folds
pc_per_trial = model.get_pc_per_trial(model.results)
ps, cs = model.get_avg_pc(pc_per_trial, N)

model.plot_all_metrics(ps, cs, pc_per_trial, model.y)




In [ ]:
purities = ps
comps = cs
all_pc = pc_per_trial
y = model.y
n = model.get_num_classes()

c_baselines, p_baselines = compute_baselines(
    model.class_counts, model.class_labels, y, n, model.class_priors)

p_intvls, c_intvls = compute_confintvls(all_pc, model.class_labels)


## Merge purity/comp average plots

Below is the code to merge the OVA and multi KDE purity/comp plots. 
To plot the binary on it owns use the plot_all_metrics in vis.py 

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from thex_data.data_consts import *
from mainmodel.helper_compute import *
from matplotlib import rc
def plot_metrics_ax(model, ax, class_metrics, xlabel, baselines=None, intervals=None):
    """
    Visualizes metric per class with bar graph; with random baseline based on class level in hierarchy.
    :param class_metrics: Mapping from class name to metric value.
    :param xlabel: Metric being plotted =  x-axis label
    :opt. param baselines: Mapping from class name to random-baseline performance (get plotted atop the bars)
    :opt. param intervals: confidence intervals, map from class 
    """
    
    thex_utils.pretty_print_dict(class_metrics, xlabel + " per class")

    class_names, metrics, baselines, intervals = get_ordered_metrics(
        class_metrics,
        baselines,
        intervals)
    # Set constants
    bar_width = 0.4
    
    errs = prep_err_bars(intervals, metrics)
    max_y = (0.4 * len(model.class_labels))
    y_indices = np.linspace(0, max_y, len(model.class_labels))

    barcolor = "#3385ff"  if xlabel == "Purity" else "#ff9933"
    # Plot bars
    ax.barh(y=y_indices, width=metrics, height=bar_width, xerr=errs,
            capsize=6,
            color=barcolor,
            edgecolor=BAR_EDGE_COLOR,
            ecolor=INTVL_COLOR)

    # Plot random baselines
    if baselines is not None:
        for index, baseline in enumerate(baselines):
            y_val = y_indices[index]
            ax.vlines(x=baseline,
                       ymin=y_val - (bar_width / 2),
                       ymax=y_val + (bar_width / 2),
                       linewidth=3,
                       linestyles=(0, (1, 1)), colors=BSLN_COLOR)

    # Format Axes, Labels, and Ticks
    ax.set_xlim(0, 1)
    x_indices, x_ticks = get_perc_ticks()
    ax.set_xticks(x_indices)
    ax.set_xticklabels(x_ticks, fontsize=16)
    ax.set_xlabel(xlabel + " (\%)", fontsize=16)
    
    
    return y_indices, class_names




def get_metrics_ax(model, axis, name):
    N = model.num_runs if model.num_runs is not None else model.num_folds
    pc_per_trial = model.get_pc_per_trial(model.results)
    ps, cs = model.get_avg_pc(pc_per_trial, N)

    c_baselines, p_baselines = compute_baselines(
        model.class_counts, model.class_labels, model.y, N, model.class_priors)
    print("purity baselines " + str(p_baselines))
    for cn in p_baselines.keys():
        print(cn + " : " +str(p_baselines[cn] ))
    p_intvls, c_intvls = compute_confintvls(pc_per_trial, model.class_labels)
    if name == "Purity":
        y_indices, class_names=plot_metrics_ax(model, axis, ps, "Purity", p_baselines, p_intvls)
    else:
        y_indices, class_names=plot_metrics_ax(model, axis, cs, "Completeness", c_baselines, c_intvls)
    
    return y_indices, class_names



In [ ]:
#  Binary

f, ax = plt.subplots(nrows=1, ncols=2,
                     sharex=True, sharey=True,
                     figsize=(6,3),  dpi=600)
rc('text', usetex=True)
mpl.rcParams['font.serif'] = ['times', 'times new roman']
mpl.rcParams['font.family'] = 'serif'

get_metrics_ax(binary_model, ax[0], "Purity")
y_indices, class_names=get_metrics_ax(binary_model, ax[1], "Completeness") 
ax[0].tick_params(direction="in")
ax[1].tick_params(direction="in")


plt.subplots_adjust(wspace=0, hspace=0)

ax[0].set_yticks(y_indices)
ax[0].set_yticklabels(clean_class_names(class_names),
           fontsize=14,  horizontalalignment='right')

plt.xticks(np.linspace(0,1,10, endpoint=False))

plt.savefig("../output/custom_figures/binary_metrics.pdf", bbox_inches='tight')
plt.show()

In [ ]:

f, ax = plt.subplots(nrows=2, ncols=2,
                     sharex=True, sharey=True,
                     figsize=(8,8),  dpi=600)
rc('text', usetex=True)
mpl.rcParams['font.serif'] = ['times', 'times new roman']
mpl.rcParams['font.family'] = 'serif'

get_metrics_ax(ova_model, ax[0][0], "Purity")
get_metrics_ax(ova_model, ax[0][1], "Completeness")

get_metrics_ax(multi_model, ax[1][0], "Purity")
y_indices, class_names=get_metrics_ax(multi_model, ax[1][1], "Completeness")

ax[0][0].tick_params(direction="in")
ax[0][1].tick_params(direction="in")

ax[0][1].text(1.05,2.1, "OVA", fontsize=20)

ax[1][1].text(1.05,2.1, "Multiclass\nKDE", fontsize=20)

plt.subplots_adjust(wspace=0, hspace=0)

ax[0][0].set_yticks(y_indices)
ax[0][0].set_yticklabels(clean_class_names(class_names),
           fontsize=16,  horizontalalignment='right')

ax[1][0].set_yticks(y_indices)
ax[1][0].set_yticklabels(clean_class_names(class_names),
           fontsize=16,  horizontalalignment='right')
plt.xticks(np.linspace(0,1,10, endpoint=False))

plt.savefig("../output/custom_figures/multis_metrics.pdf", bbox_inches='tight')
plt.show()

## Plot probability plots together

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from models.binary_model.binary_model import BinaryModel
from models.ind_model.ind_model import OvAModel
from models.multi_model.multi_model import MultiModel
from mainmodel.helper_compute import get_perc_ticks, clean_class_name
from thex_data.data_consts import *
from utilities import utilities as thex_utils
from matplotlib import rc
import matplotlib as mpl
import pickle


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl
from thex_data.data_consts import *
from utilities import utilities as thex_utils


indices = [0, 1, 3, 7, 10]
rc('text', usetex=True)
cur_model = multi_model
class_labels = cur_model.class_labels 

num_classes = len(indices)

f, ax = plt.subplots(nrows=len(indices),
                     ncols=2,
                     sharex=True, sharey=True,
                     figsize=(5, 10),
                     dpi=DPI)
row_index = 0
for index in indices:
    if row_index == 0:
            # Add titles to top  
        ax[row_index][1].set_title("OVA", fontsize=16)
        ax[row_index][0].set_title("Multi", fontsize=16)
    cn = cur_model.class_labels[index]
    plot_model_rates(cn, cur_model, ax[row_index][0])  
    ax[row_index][col_index].text(-0.45, 0.81, clean_class_name(cn), fontsize=18)
    plot_model_rates(cn, ova_model, ax[row_index][1])
    row_index+=1
    
y_indices = [0.1, 0.3, 0.5, 0.7, 0.9]
y_ticks = ["10", "30", "50", "70", "90"]
# x and y indices/ticks are the same
plt.xticks(np.arange(5), y_ticks)
plt.yticks(y_indices, y_ticks)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

mpl.rcParams['font.serif'] = ['times', 'times new roman']
mpl.rcParams['font.family'] = 'serif'

f.text(0.5, 0.08, 'Assigned Probability ' + r' $\pm10\%$', fontsize=14, ha='center')
f.text(0.03, .5, r'Empirical Probability $\equiv$ TP/Total ($\%$)',
       fontsize=14, va='center', rotation='vertical')
plt.subplots_adjust(wspace=0, hspace=0)
f.savefig(ROOT_DIR + "/output/custom_figures/prob_callouts.pdf", bbox_inches='tight')
plt.show()

In [ ]:

# call outs for multi only
indices = [0, 1, 3, 7, 10]
rc('text', usetex=True)
cur_model = multi_model
class_labels = cur_model.class_labels 

num_classes = len(indices)

f, ax = plt.subplots(nrows=1,
                     ncols=len(indices),
                     sharex=True, sharey=True,
                     figsize=(8, 2),
                     dpi=280)
row_index = 0
for index in indices: 
    cn = cur_model.class_labels[index]
    plot_model_rates(cn, cur_model, ax[row_index])  
    ax[row_index].text(-0.45, 0.81, clean_class_name(cn), fontsize=14)
#     plot_model_rates(cn, ova_model, ax[row_index][1])
    row_index+=1
    
y_indices = [0.1, 0.3, 0.5, 0.7, 0.9]
y_ticks = ["10", "30", "50", "70", "90"]
# x and y indices/ticks are the same
plt.xticks(np.arange(5), y_ticks)
plt.yticks(y_indices, y_ticks)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

mpl.rcParams['font.serif'] = ['times', 'times new roman']
mpl.rcParams['font.family'] = 'serif'

f.text(0.5, -0.07, 'Assigned Probability ' + r' $\pm10\%$', fontsize=14, ha='center')
f.text(0.05, .5, r'Empirical Prob. ($\%$)',
       fontsize=14, va='center', rotation='vertical')
plt.subplots_adjust(wspace=0, hspace=0)
f.savefig(ROOT_DIR + "/output/custom_figures/prob_callouts.pdf", bbox_inches='tight')
plt.show()

In [ ]:
ova_model.range_metrics = ova_model.compute_probability_range_metrics(
        ova_model.results, bin_size=0.2)
binary_model.range_metrics = binary_model.compute_probability_range_metrics(
        binary_model.results, bin_size=0.2)
multi_model.range_metrics = multi_model.compute_probability_range_metrics(
        multi_model.results, bin_size=0.2)




In [ ]:
plot_rates_together(binary_model, ova_model, multi_model, indices=[0,1,2,3,4,5])

plot_rates_together(binary_model, ova_model, multi_model, indices=[6,7,8,9,10,11])

In [ ]:
ova_model.range_metrics = ova_model.compute_probability_range_metrics(
        ova_model.results, bin_size=0.1)
binary_model.range_metrics = binary_model.compute_probability_range_metrics(
        binary_model.results, bin_size=0.1)
multi_model.range_metrics = multi_model.compute_probability_range_metrics(
        multi_model.results, bin_size=0.1)

In [ ]:
[0, 0.1,0.2,.3,.4,.5,.6,.7,8.]

In [ ]:
np.linspace(0,1,11)[:-1]

In [ ]:
def plot_model_curves(class_name, model, ax):
    """
    Plots rates for this model/class on axis, with annotations
    """
    purities, comps = get_pc_per_range(model, class_name)

    def plot_axis(ax, data, color):
        """
        Plot data on axis in certain color
        """
        x_indices = np.linspace(0, 1, 11)[:-1]

        ax.scatter(x_indices, data, color=color, s=4)
        print("Data: " + str(data))
        ax.plot(x_indices, data, color=color, linewidth=2)
        ax.set_yticks([])  # same for y ticks
        ax.set_ylim([0, 1])

    print("\n\n P-C metrics for : " + class_name)
    plot_axis(ax, comps, C_BAR_COLOR)
    ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
    ax2.set_ylim([0, 1])
    plot_axis(ax2, purities, P_BAR_COLOR)
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    return ax2


def plot_pc_curves_together(binary_model, ova_model, multi_model, indices=None):
    """
    Plot class versus probability rates of all three classifiers together
    :param indices: class indices to plot
    """
    class_labels = ova_model.class_labels
    num_classes = len(ova_model.class_labels)
    if indices is not None:
        num_classes = len(indices)
    f, ax = plt.subplots(nrows=num_classes,
                         ncols=3,
                         sharex=True, sharey=True,
                         figsize=(FIG_WIDTH, 10),
                         dpi=DPI)

    y_indices = [0, 0.2, 0.4, 0.6, 0.8, 1]
    y_ticks = ["0", "20", "40", "60", "80", ""]

    plot_index = 0
    for class_index in range(len(class_labels)):
        if indices is not None and class_index not in indices:
            continue

        if plot_index == 0:
            # Add titles to top of plots
            ax[plot_index][0].set_title("Binary", fontsize=TICK_S)
            ax[plot_index][1].set_title("OVA", fontsize=TICK_S)
            ax[plot_index][2].set_title("Multi", fontsize=TICK_S)

        class_name = class_labels[class_index]
        plot_model_curves(class_name, binary_model, ax[plot_index][0])

        plot_model_curves(class_name, ova_model, ax[plot_index][1])
        mirror_ax = plot_model_curves(class_name, multi_model, ax[plot_index][2])

        ax[plot_index][0].set_yticks(ticks=y_indices)
        ax[plot_index][0].set_yticklabels(labels=y_ticks, color=P_BAR_COLOR)
        mirror_ax.set_yticks(ticks=y_indices)
        mirror_ax.set_yticklabels(labels=y_ticks, color=C_BAR_COLOR)
        ax[plot_index][0].tick_params(axis='both', direction='in', labelsize=10)
        ax[plot_index][1].tick_params(axis='both', direction='in')
        ax[plot_index][2].tick_params(axis='both', direction='in', labelsize=10)

        mpl.rcParams['font.serif'] = ['times', 'times new roman']
        mpl.rcParams['font.family'] = 'serif'
        pretty_class_name = clean_class_name(class_name)
        ax[plot_index][0].text(0, 0.85, pretty_class_name, fontsize=14)
        plot_index += 1

    x_indices = np.linspace(0, 1, 11)[:-1]

    plt.xticks(x_indices, ["", "10", "", "30", "", "50", "", "70", "", "90"])
    rc('text', usetex=True)
    f.text(0.5, 0.08, r'Probability $\geq$X\%', fontsize=TICK_S, ha='center')
    f.text(0.03, .5, 'Purity (\%)',
           fontsize=TICK_S, va='center', rotation='vertical', color=P_BAR_COLOR)
    f.text(0.98, .5, 'Completeness (\%)',
           fontsize=TICK_S, va='center', rotation='vertical', color=C_BAR_COLOR)

    plt.subplots_adjust(wspace=0, hspace=0)

    f.savefig("../output/custom_figures/merged_pc_curves_" +
              str(indices) + ".pdf", bbox_inches='tight')
    plt.show()

In [ ]:

plot_pc_curves_together(binary_model, ova_model, multi_model, indices=[0,1,2,3,4,5])
plot_pc_curves_together(binary_model, ova_model, multi_model, indices=[6,7,8,9,10,11])
plot_pc_curves_together(binary_model, ova_model, multi_model, indices=[0,7,8,9,10,11])


## Combining with vs without priors


In [ ]:
labels = ["Unspecified Ia", "Unspecified II", "Ia-91bg", "TDE", "Ib", "Ic", "Ib/c", "Unspecified Ib",  "IIb"]
model = MultiModel(folds=10,
                   min_class_size = 40,  
                   class_labels = labels,
                   priors = True,
                   transform_features = True,
                   cols = mags,
                   lsst_test=True)  
multi_w = load_prev_exp(exp_dir, "multi_priors_w_ibc_total/Multiclass_Classifier2", model=model)


modelwithout = MultiModel(folds=10,
                   min_class_size = 40,  
                   class_labels = labels,
                   transform_features = True,
                   cols = mags,
                   lsst_test=True)  
multi_wo = load_prev_exp(exp_dir,"multi_no_priors_w_ibc_total/Multiclass_Classifier1",model=modelwithout)


In [ ]:
from mainmodel.helper_compute import *
from thex_data.data_consts import * 


def get_model_stats(model):
    N = model.num_runs if model.num_runs is not None else model.num_folds
    pc_per_trial = model.get_pc_per_trial(model.results)
    ps, cs = model.get_avg_pc(pc_per_trial, N)
    c_baselines, p_baselines = compute_baselines(
            model.class_counts, model.class_labels, model.y, len(model.class_labels), model.class_priors)
    p_intvls, c_intvls = compute_confintvls(pc_per_trial, model.class_labels)
    
    p_class_names, p_metrics, p_b, p_intvls = get_ordered_metrics(
            ps,
            p_baselines,
            p_intvls)
    
    c_class_names, c_metrics, c_b, c_intvls = get_ordered_metrics(
            cs,
            c_baselines,
            c_intvls)
    
    m_stats = {"Purity":[p_class_names,p_metrics,p_b,p_intvls],
              "Completeness":[c_class_names, c_metrics, c_b, c_intvls]}
    return m_stats
         
def plot_m(ax, indices, errs, baselines, metrics, name, color):

    bar_width=0.1
    ax.barh(y=indices, 
            width=metrics, 
            height=bar_width, 
            xerr=errs,
            capsize=7, 
            linewidth=0.5,
            edgecolor=BAR_EDGE_COLOR, 
            ecolor=INTVL_COLOR, 
            color=color,
            label=name)
    for index, baseline in enumerate(baselines):
        y_val = indices[index]
        ax.vlines(x=baseline,
                   ymin=y_val - (bar_width / 2),
                   ymax=y_val + (bar_width / 2),
                   linewidth=2,
               linestyles=(0, (1, 1)), colors=BSLN_COLOR)
            
def plot_priors_together(axis, WP_model, WP_model_stats, NP_model, NP_model_stats, plot_type):
    m1_class_names, m1_metrics, m1_b, m1_intvls  = WP_model_stats[plot_type]
    m2_class_names, m2_metrics, m2_b, m2_intvls  = NP_model_stats[plot_type]
    WITHOUT_COLOR = "#808080" # without 
    WITH_COLOR = "#80b3ff" # with
        
    m1_errs = prep_err_bars(m1_intvls, m1_metrics) 
    m1_indices =[0.1, 0.3, 0.5, 0.7, 0.9]
    
    # Print performance per class Plus/minus 95% confidence
    print("\n"+plot_type+" stats:")
    for index, cn in enumerate(m1_class_names):
        cur_met_W= m1_metrics[index]
        d_W = (m1_intvls[index][1]-m1_intvls[index][0])/2
        print(cn+ " WP : " + str(round(cur_met_W,3)*100) + "\% \pm " + str(round(d_W,4)*100) +"\%")
        
        cur_met_WO = m2_metrics[index]
        d_WO = (m2_intvls[index][1]-m2_intvls[index][0])/2
        print(cn+ " NP : " + str(round(cur_met_WO,3)*100) + "\% \pm " + str(round(d_WO,4)*100) +"\%")
        
        # calculate if significant
        if cur_met_W>cur_met_WO:
            if cur_met_W-d_W > cur_met_WO+d_WO:
                print("SIGNIFICANT")
        else:
            if cur_met_W+d_W < cur_met_WO-d_WO:
                print("SIGNIFICANT")
        
    plot_m(ax=axis, indices=m1_indices, errs=m1_errs, baselines=m1_b, 
           metrics=m1_metrics, color=WITH_COLOR, name=WP_model.name)
    m2_errs = prep_err_bars(m2_intvls, m2_metrics)
    plot_m(ax=axis, indices=[0.2,0.4,0.6,0.8,1], errs=m2_errs, baselines=m2_b, 
           metrics=m2_metrics, color=WITHOUT_COLOR, name=NP_model.name)
    # Figure formatting
    axis.set_xlim(0, 1)
    axis.set_xticks([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1])
    axis.set_xticklabels(["0", "", "20", "", "40", "", "60", "", "80", "", ""], fontsize=14)
    axis.set_xlabel(plot_type+" (\%)", fontsize=16)  
    if plot_type == "Purity": 
        ylabel_points = [0.15, 0.35, 0.55, 0.75, 0.95]
        axis.set_yticks(ylabel_points)
        axis.set_yticklabels(clean_class_names(m1_class_names),  fontsize=16,
                       horizontalalignment='right')
    else:
        axis.legend(fontsize=11, loc="lower right", labelspacing=.2, handlelength=1)


f, ax = plt.subplots(nrows=1, ncols=2,
                     sharex=True, sharey=True,
                     figsize=(6,3),  dpi=400)

rc('text', usetex=True)
mpl.rcParams['font.serif'] = ['times', 'times new roman']
mpl.rcParams['font.family'] = 'serif'

WP_stats =  get_model_stats(multi_w) # WP = with priors
NP_stats = get_model_stats(multi_wo) # NP = no priors
multi_w.name = "Freq-based priors"
multi_wo.name = "Uniform priors"

plot_priors_together(axis=ax[0],
                     WP_model=multi_w, 
                     WP_model_stats=WP_stats, 
                     NP_model=multi_wo, 
                     NP_model_stats=NP_stats, 
                     plot_type="Purity")
plot_priors_together(axis=ax[1],
                     WP_model=multi_w, 
                     WP_model_stats=WP_stats, 
                     NP_model=multi_wo, 
                     NP_model_stats=NP_stats, 
                     plot_type="Completeness")

plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("../output/custom_figures/prior_comp_combined.pdf", bbox_inches='tight')
plt.show() 

#### combine probability plots for w priors vs without priors

In [ ]:
multi_w.range_metrics = multi_w.compute_probability_range_metrics(
        multi_w.results, bin_size=0.2)
multi_wo.range_metrics = multi_wo.compute_probability_range_metrics(
        multi_wo.results, bin_size=0.2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from thex_data.data_consts import ROOT_DIR
from mainmodel.helper_plotting import *

rc('text', usetex=True) 
mpl.rcParams['font.serif'] = ['times', 'times new roman']
mpl.rcParams['font.family'] = 'serif'

class_labels = multi_w.class_labels
class_labels = ["Unspecified Ia", "Unspecified II", "Ia-91bg", "TDE", "Ib/c"]
num_classes = len(multi_w.class_labels) 
f, ax = plt.subplots(nrows=num_classes,
                     ncols=2,
                     sharex=True, sharey=True,
                     figsize=(FIG_WIDTH, 8),
                     dpi=DPI)
plot_index = 0
for class_index in range(len(class_labels)):
    if plot_index == 0:
        # Add titles to top of plots
        ax[plot_index][0].set_title("Frequency-based Priors", fontsize=14)
        ax[plot_index][1].set_title("Uniform Priors", fontsize=14)

    class_name = class_labels[class_index]  
    plot_model_rates(class_name, multi_w, ax[plot_index][0])
    plot_model_rates(class_name, multi_wo, ax[plot_index][1])
    pretty_class_name = clean_class_name(class_name)
    ax[plot_index][0].text(-0.2, 0.85, pretty_class_name, fontsize=16) 
    plot_index += 1

y_indices = [0.1, 0.3, 0.5, 0.7, 0.9]
y_ticks = ["10", "30", "50", "70", "90"]
# x and y indices/ticks are the same
plt.xticks(np.arange(5), y_ticks)
plt.yticks(y_indices, y_ticks)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)

f.text(0.5, 0.06, 'Assigned Probability' + r' $\pm$10\%', fontsize=14, ha='center')
f.text(0.02, .5, r'Empirical Probability $\equiv$ TP/Total ($\%$)',
           fontsize=14, va='center', rotation='vertical')

plt.subplots_adjust(wspace=0, hspace=0)

f.savefig(ROOT_DIR + "/output/custom_figures/merged_metrics_priors_comp.pdf", bbox_inches='tight')
plt.show()

## Example output plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from mainmodel.helper_compute import *
from thex_data.data_consts import *
import utilities.utilities as thex_utils

model = MultiModel(folds = 3,
                   min_class_size = 40,  
#                    class_labels = ['Unspecified Ia', 'Unspecified II', 'Ia-91bg', 'TDE'],
#                    priors = [0.73, 0.24, 0.01, 0.007],
                   transform_features = True,
                   cols = mags)  

model.run_model()

In [ ]:
model.class_labels

In [ ]:
model.results[0].shape

In [ ]:
#  use following to find good samples.
for do_index in range(0,3407):
    row = model.results[0][do_index] 
    labels = row[len(row) - 1]
    true_class_index = None

    for class_index, class_name in enumerate(model.class_labels):
        if class_name in thex_utils.convert_str_to_list(labels):
            true_class_index = class_index 
#     if model.class_labels[true_class_index] != "Unspecified II" and model.class_labels[true_class_index] != "Unspecified Ia":

    if model.class_labels[true_class_index] == "Unspecified Ia":
        print(do_index)

Want to find 16 quality examples

8: good & correct examples with wide dist
    correct examples for II P, IIn, TDE, GRB, Ib (unspec.)

4: not 'good' but correct examples (high prob on Ia or something)

4: incorrect, but benefit from empirical prob plots (so look for maybe Ia with probs 20-49%, where Ia IS the correct answer but something else won; or could show 4 GRB plots with probs >80%, and half are correct but half are not- this exemplifies its 50% empriical prob at the range. 



In [ ]:
a=[range(200)]
import numpy as np
indices=np.arange(109, 400)
 
# 
for index in indices:
    f, ax = plt.subplots(nrows=1, ncols=1,  figsize=(4,2), dpi=DPI)
    plot_on_ax(ax, model, index) 
    plt.savefig("../output/custom_figures/sample_" + str(index) + ".pdf")

In [ ]:
model.class_labels

In [ ]:
def plot_on_ax(axis, model, do_index, example_num):
    """
    Plot example on given ax
    """
    row = model.results[0][do_index] 
    labels = row[len(row) - 1]
    true_class_index = None

    for class_index, class_name in enumerate(model.class_labels):
        if class_name in thex_utils.convert_str_to_list(labels):
            true_class_index = class_index


    ACC = "#ccccff"# actual class color, light blue
    DCC = "#0000cc" # default class color, dark blue

    colors = [DCC] * len(model.class_labels)
    colors[true_class_index] = ACC
    probabilities = row[0:len(row) - 1]  
    bar_size = 0.05
    x_indices = np.linspace(0,
                            len(model.class_labels) * bar_size,
                            len(model.class_labels))

    axis.bar(x=x_indices, height=probabilities,  
             width=bar_size, 
             color=colors, edgecolor='black')
    axis.tick_params(axis="y",direction="in")
    axis.set_ylim([0, 1])
    
#     don't overlap # on bar; this applies to TDE.
    text_xloc = 0.5
    if true_class_index == 10:
        text_xloc = 0.45
    axis.text(text_xloc, 0.8,  str(example_num), style='italic',fontsize=15)
    



In [ ]:
# right
# 2370 (ib), 3359  3367  3368 (TDE),  3367
# 3374, 3375,  (GRB)
# 3148, 3180 (II P)
#  254,  (Ia)
# 126, 263 (typical Ia)

# wrong
#  
#  3390, 3398, 3404, GRB 

# wrong, but prob is low so you shouldn't believe it anyways
# 3360 (TDE)
#  3183, 3199, 3207 (II P) 
# 156, 216 Ia


indices = [[2370, 3359, 3375, 3148],  # Good
          [3148, 3180, 263, 254], # good too 
          [3360, 3199, 3360, 3183], # wrong, low probs
          [ 3207, 3390,  3398, 3404]] #wrong but empirical probs would help a lot.
    
f, ax = plt.subplots(nrows=4,
                     ncols=4,
                     sharex=True, sharey=True,
                     figsize=(8,7),
                     dpi=DPI)
do_index = 0
example_num = 1
for row_index, index_set in enumerate(indices): 
    cur_ax = ax[row_index]
    
    for plot_index, do_index in enumerate(index_set): 
        plot_on_ax(cur_ax[plot_index], model, do_index, example_num)
        example_num+=1

        if plot_index==0:
            yticks = np.arange(0,1.2,.2)
            cur_ax[plot_index].set_yticks(ticks=yticks)
            cur_ax[plot_index].set_yticklabels(labels=[str(int(i*100)) for i in yticks],
                                               fontsize=12)


f.text(.04, 0.4, 'Probability (%)', fontsize=14, ha='center', rotation =90)

bar_size = 0.05
x_indices = np.linspace(0, len(model.class_labels) * bar_size,
                            len(model.class_labels))


pretty_class_names = clean_class_names(model.class_labels) 
xticksize = 11
ax[3][0].set_xticks(ticks=x_indices)
ax[3][0].set_xticklabels(labels=pretty_class_names, fontsize=xticksize, rotation = -90)
ax[3][1].set_xticks(ticks=x_indices)
ax[3][1].set_xticklabels(labels=pretty_class_names, fontsize=xticksize, rotation = -90)
ax[3][2].set_xticks(ticks=x_indices)
ax[3][2].set_xticklabels(labels=pretty_class_names, fontsize=xticksize, rotation = -90)
ax[3][3].set_xticks(ticks=x_indices)
ax[3][3].set_xticklabels(labels=pretty_class_names, fontsize=xticksize, rotation = -90)


plt.subplots_adjust(wspace=0, hspace=0, left=0.1)

plt.savefig("../output/custom_figures/examples.pdf", bbox_inches='tight')
plt.show()




In [ ]:
model.y.shape

In [ ]:
import pickle
with open('../output/custom_figures/full_model_data.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)